In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
from urllib.request import Request
import requests

import time
from tqdm.notebook import tqdm

import re
import pandas as pd


### 로그인 정보 쿠키 가져오기

In [5]:
chrome_options = webdriver.ChromeOptions()

url = 'https://boardgamegeek.com/login'

def requests_cookie(url) :
    response = requests.get(url)
    print(response.headers)
    print(response.cookies)
    soup = BeautifulSoup(response.text, 'html.parser')
    print(soup)
    
requests_cookie(url)

#아이디와 비밀번호를 *****대신 입력하세요
nickname = '**********'
password = '**********'

def reuse_cookies_test(url) :
    options = chrome_options
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    
    driver = webdriver.Chrome('../driver/chromedriver', chrome_options=chrome_options)
    driver.implicitly_wait(5)
    driver.get(url)
    
    # 로그인
    id_slot = driver.find_element(By.ID, "inputUsername")
    pw_slot = driver.find_element(By.ID, "inputPassword")
    id_slot.send_keys(nickname)
    pw_slot.send_keys(password)
    pw_slot.send_keys(Keys.ENTER)
    
    driver.implicitly_wait(5)
    time.sleep(2)
    
    print(driver.get_cookies)
    time.sleep(2)
    
    # 쿠키(로그인 여부) 저장
    _cookies = driver.get_cookies()
    cookie_dict = {}
    
    for cookie in _cookies :
        cookie_dict[cookie['name']] = cookie['value']
        
    print(cookie_dict)
    return cookie_dict
    
cookie_dict = reuse_cookies_test(url)

{'Server': 'nginx/1.19.10', 'Date': 'Fri, 10 Feb 2023 04:08:54 GMT', 'Content-Type': 'text/html; charset="UTF-8"', 'X-Powered-By': 'PHP/7.4.14', 'Set-Cookie': 'SessionID=13d0f9abf75f68bfd8d34521b2f031e82418711c; expires=Fri, 10-Feb-2023 05:08:54 GMT; Max-Age=3600; path=/; secure; HttpOnly; SameSite=Lax', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate', 'Pragma': 'no-cache', 'Content-Security-Policy': "frame-ancestors 'self'", 'X-Frame-Options': 'sameorigin', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'X-Varnish': '1003325123', 'Age': '0', 'Accept-Ranges': 'bytes', 'Via': '1.1 varnish (Varnish/6.6), 1.1 google', 'Strict-Transport-Security': 'max-age=31536000', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'Transfer-Encoding': 'chunked'}
<RequestsCookieJar[<Cookie SessionID=13d0f9abf75f68bfd8d34521b2f031e82418711c for boardgamegeek.com/>]>
<!DOCTYPE html>

<html lang="en-US">
<head>
<link href="https://cf.geekd

/tmp/ipykernel_3345/2550992826.py:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('../driver/chromedriver', chrome_options=chrome_options)
/tmp/ipykernel_3345/2550992826.py:23: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('../driver/chromedriver', chrome_options=chrome_options)


<bound method WebDriver.get_cookies of <selenium.webdriver.chrome.webdriver.WebDriver (session="8b7f073f6d06ec654293458f521ff7c9")>>
{'__gpi': 'UID=00000bbed74d0e57:T=1676002137:RT=1676002137:S=ALNI_MZ9IrhXy1NtJtdJ9nqPQnIDFjmnXA', '__gads': 'ID=6979b6605156b3b2:T=1676002137:S=ALNI_MbgRnW6Y4fsQlnqo0ndwGCSZ_hGXg', '_ga': 'GA1.2.1488961772.1676002136', '_ga_GMNMCY4DVM': 'GS1.1.1676002136.1.1.1676002137.0.0.0', 'SessionID': 'cd6d9221cdae48ced91227d23fb3c5bae602e37fu3453868', 'bggusername': 'yongho0166', '_gat_gtag_UA_104725_1': '1', 'bggpassword': 'co7kghrli5h2t72103ly1fpr2gohsscwx', '_gid': 'GA1.2.1529528288.1676002136'}


In [6]:
session = requests.Session()
print(session.cookies)

session.cookies.update(cookie_dict)
print(session.cookies)

<RequestsCookieJar[]>
<RequestsCookieJar[<Cookie SessionID=cd6d9221cdae48ced91227d23fb3c5bae602e37fu3453868 for />, <Cookie __gads=ID=6979b6605156b3b2:T=1676002137:S=ALNI_MbgRnW6Y4fsQlnqo0ndwGCSZ_hGXg for />, <Cookie __gpi=UID=00000bbed74d0e57:T=1676002137:RT=1676002137:S=ALNI_MZ9IrhXy1NtJtdJ9nqPQnIDFjmnXA for />, <Cookie _ga=GA1.2.1488961772.1676002136 for />, <Cookie _ga_GMNMCY4DVM=GS1.1.1676002136.1.1.1676002137.0.0.0 for />, <Cookie _gat_gtag_UA_104725_1=1 for />, <Cookie _gid=GA1.2.1529528288.1676002136 for />, <Cookie bggpassword=co7kghrli5h2t72103ly1fpr2gohsscwx for />, <Cookie bggusername=yongho0166 for />]>


### 보드게임 목록 페이지에서 크롤링
- 이름, 출시연도, 상세페이지 url, geek평점, 평균평점

In [4]:
name = []
year = []
url_detail = []
geek_rating = []
avg_rating = []

url = 'https://boardgamegeek.com/browse/boardgame/page/{page_num}'


for a in tqdm(range(1,240)) : 
    url1 = url.format(page_num = urllib.parse.quote(str(a)))
    res = session.get(url1)
    html = res.text
    soup = BeautifulSoup(html, "html.parser")
    
    box = soup.find_all('tr', id = 'row_')
    
    print('----------------')
    print(a)
    print('----------------')
    
    for i in box : 
        rating = i.find_all(class_ = 'collection_bggrating')
        if rating[0].string == "\n\t\t\tN/A\t\t" :
            break
        name.append(i.find(id = re.compile('results_objectname')).find('a').get_text())
        try : 
            year.append(i.find(id = re.compile('results_objectname')).find('span').get_text())
        except :
            year.append("None")
        
        url_detail.append(i.find(id = re.compile('results_objectname')).find('a')['href'])

        
        geek_rating.append(float(rating[0].string))
        avg_rating.append(float(rating[1].string))

  0%|          | 0/239 [00:00<?, ?it/s]

----------------
1
----------------
----------------
2
----------------
----------------
3
----------------
----------------
4
----------------
----------------
5
----------------
----------------
6
----------------
----------------
7
----------------
----------------
8
----------------
----------------
9
----------------
----------------
10
----------------
----------------
11
----------------
----------------
12
----------------
----------------
13
----------------
----------------
14
----------------
----------------
15
----------------
----------------
16
----------------
----------------
17
----------------
----------------
18
----------------
----------------
19
----------------
----------------
20
----------------
----------------
21
----------------
----------------
22
----------------
----------------
23
----------------
----------------
24
----------------
----------------
25
----------------
----------------
26
----------------
----------------
27
----------------
----------

----------------
220
----------------
----------------
221
----------------
----------------
222
----------------
----------------
223
----------------
----------------
224
----------------
----------------
225
----------------
----------------
226
----------------
----------------
227
----------------
----------------
228
----------------
----------------
229
----------------
----------------
230
----------------
----------------
231
----------------
----------------
232
----------------
----------------
233
----------------
----------------
234
----------------
----------------
235
----------------
----------------
236
----------------
----------------
237
----------------
----------------
238
----------------
----------------
239
----------------


In [6]:
len(name), len(year), len(url_detail), len(geek_rating), len(avg_rating)

(23900, 23900, 23900, 23900, 23900)

In [7]:
boardgame_data_df = pd.DataFrame({
    "이름" : name,
    "출시일" : year,
    "url" : url_detail,
    "geek평점" : geek_rating,
    "평균평점" : avg_rating
    
})

In [8]:
boardgame_data_df.to_csv("../data/boardgame_data_new.csv", sep = "," , encoding = "utf-8")

### 각 보드게임 상세 정보 페이지에 들어가서 크롤링
- 전체 순위, 최소/최대 플레이 인원, 최소/최대 플레이 시간, 적정 연령, 난이도, 진행방식, 카테고리
- 동적페이지로 구성되어서 javascript를 직접 정규식으로 크롤링

In [5]:
rank_list = []
min_player_list = []
max_player_list = []
min_playtime_list = []
max_playtime_list = []
player_age_list = []
average_weight_list = []
boardgamemechanic_fulllist = []
boardgamecategory_fulllist = []


url2 = "https://boardgamegeek.com"
for i in tqdm(url_detail) :
    url3 = url2 + i + '/credits'
    res = session.get(url3)
    html = res.text
    soup = BeautifulSoup(html, "html.parser")
    
    script = soup.find_all('script')[2].text

    #rank 정규식
    p1 = re.compile("Overall\sRank[^\}]*\"rank\":\"\d+\"")
    rank_str = p1.search(script).group()

    p2 = re.compile("\"rank\":\"\d+\"")
    rank = p2.search(rank_str).group()
    rank = rank.replace("\"rank\":","").replace("\"","")
    rank_list.append(rank)
    
    #minplayer
    p1 = re.compile("\"versioninfo\".*\"maxplaytime\":\"\d+\"")
    player_time = p1.search(script).group()

    p2 = re.compile("\"minplayers\":\"\d+\"")
    min_player = p2.search(player_time).group()
    min_player = min_player.replace("\"minplayers\":","").replace("\"","")
    min_player_list.append(min_player)

    #maxplayer
    p2 = re.compile("\"maxplayers\":\"\d+\"")
    max_player = p2.search(player_time).group()
    max_player = max_player.replace("\"maxplayers\":","").replace("\"","")
    max_player_list.append(max_player)

    #minplaytime
    p2 = re.compile("\"minplaytime\":\"\d+\"")
    min_playtime = p2.search(player_time).group()
    min_playtime = min_playtime.replace("\"minplaytime\":","").replace("\"","")
    min_playtime_list.append(min_playtime)
    
    p2 = re.compile("\"maxplaytime\":\"\d+\"")
    max_playtime = p2.search(player_time).group()
    max_playtime = max_playtime.replace("\"maxplaytime\":","").replace("\"","")
    max_playtime_list.append(max_playtime)
    
    #player age
    p1 = re.compile("\"polls\".*\"votes\"")
    age_weight = p1.search(script).group()

    p2 = re.compile("\"playerage\":\"[^\"]*")
    player_age = p2.search(age_weight).group()
    player_age= player_age.replace("\"playerage\":","").replace("\"","")
    player_age_list.append(player_age)

    #average_weight
    p2 = re.compile("\"averageweight\":.*\,")
    average_weight = p2.search(age_weight).group()
    average_weight = average_weight.replace("\"averageweight\":","").replace(",","")
    average_weight_list.append(average_weight)
    
    #Mechanisms
    try : 
        p1 = re.compile('\"boardgamemechanic\":\[[^\]]+\]')
        boardgamemechanics = p1.search(script).group()

        p2 = re.compile('\"name\":\"[^\"]+\"')
        boardgamemechanic_list = p2.findall(boardgamemechanics)
        boardgamemechanic_list = [boardgamemechanic[8:].strip().replace('\\','').replace('\"','') for boardgamemechanic in boardgamemechanic_list]
        boardgamemechanic_fulllist.append(boardgamemechanic_list)
    except : 
        boardgamemechanic_fulllist.append([])
    #Categories
    try : 
        p1 = re.compile('\"boardgamecategory\":\[[^\]]+\]')
        boardgamecategories = p1.search(script).group()

        p4 = re.compile('\"name\":\"[^\"]+\"')
        boardgamecategory_list = p2.findall(boardgamecategories)
        boardgamecategory_list = [boardgamecategory[8:].strip().replace('\\','').replace('\"','') for boardgamecategory in boardgamecategory_list]
        boardgamecategory_fulllist.append(boardgamecategory_list)
    except :
        boardgamecategory_fulllist.append([])

  0%|          | 0/23900 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
len(rank_list),len(min_player_list),len(max_player_list),len(min_playtime_list),len(max_playtime_list),len(player_age_list),len(average_weight_list),len(boardgamemechanic_fulllist),len(boardgamecategory_fulllist)

(23900, 23900, 23900, 23900, 23900, 23900, 23900, 23900, 23900)

In [12]:
boardgame_detaildata_df = pd.DataFrame({
    "랭크" : rank_list,
    "최소인원" : min_player_list,
    "최대인원" : max_player_list,
    "최소게임시간" : min_playtime_list,
    "최대게임시간" : max_playtime_list,
    "적정나이" : player_age_list,
    "난이도" :average_weight_list,
    "진행방식" : boardgamemechanic_fulllist,
    "종류" : boardgamecategory_fulllist})


In [13]:
boardgame_detaildata_df.to_csv("../data/boardgame_detaildata_new.csv", sep = "," , encoding = "utf-8")

### 가격정보 가져오기
- selenium 사용

In [7]:
chrome_options = webdriver.ChromeOptions()

url = 'https://boardgamegeek.com/browse/boardgame'
driver = webdriver.Chrome(service=Service('../driver/chromedriver'), options = chrome_options)
driver.get(url)

In [11]:
signin_bt = driver.find_element(By.XPATH, '//*[@id="global-header-outer"]/header/nav/div/div[2]/div/div[1]/ul/li[7]/button')
signin_bt.click()

In [12]:
userid_slot = driver.find_element(By.ID, 'inputUsername')
userpassword_slot = driver.find_element(By.ID, 'inputPassword')
userid_slot.send_keys('yongho0166')
userpassword_slot.send_keys('qlalfqjsgh2357')

In [13]:
signin_bt2 = driver.find_element(By.CLASS_NAME, 'btn.btn-primary')
signin_bt2.click()

In [14]:
name_plus = []
price = []
url_detail2 = []

for a in tqdm(range(239)) :
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    box = soup.find_all('tr', id = 'row_')
    
    for i in box :
        name_plus.append(i.find(id = re.compile('results_objectname')).find('a').get_text())
        price_data_box =  i.find(class_ = 'collection_shop')
        try :
            price_all = price_data_box.find_all("span")
            price_data = price_all[-1].string.replace('$','')
            if price_data == '(unavailable)' :
                price.append('-1')
            elif price_data == 'To low to display':
                price.append('-1')
            else :
                price.append(price_data)
        except :
            price.append('-1')
        
        url_detail2.append(i.find(id = re.compile('results_objectname')).find('a')['href'])

    page_box = driver.find_element(By.CLASS_NAME,'infobox')
    page_slot = page_box.find_elements(By.TAG_NAME, 'a')
    for i in page_slot :
        if i.get_attribute('title') == 'next page' :
            i.click()
            driver.implicitly_wait(10)
            break

  0%|          | 0/239 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [29]:
len(name_plus), len(price), len(url_detail2)

(23900, 23900, 23900)

In [33]:
driver.close()

In [30]:
boardgame_price_df = pd.DataFrame({
    "이름" : name_plus,
    "가격" : price,
    "url" : url_detail2
})


In [31]:
boardgame_price_df.tail()

,이름,가격,url
23895,LCR,7.58,/boardgame/3522/lcr
23896,The Game of Life,21.99,/boardgame/2921/game-of-life
23897,Trouble,10.52,/boardgame/1410/trouble
23898,Monopoly,23.95,/boardgame/1406/monopoly
23899,War,-1,/boardgame/16398/war


In [32]:
#추가 옵션 index = False 를 넣으면 쓰레기값 없어진다
boardgame_price_df.to_csv("../data/boardgame_price_df_new.csv", sep = "," , encoding = "utf-8",index = False)